In [1]:
import requests
from bs4 import BeautifulSoup
import sys
import pandas as pd
import numpy as np
import re
import csv
import unicodedata
import sys
import time

Additional Scrapping was done on script sent to AWS

### Scraping Part 1
Get links to each beer in the Top 100 beer list per category

In [2]:
# GET DATA
url_amer_ipa = 'https://www.beeradvocate.com/lists/style/116/'
url_ne_ipa = 'https://www.beeradvocate.com/lists/style/189/'
url_imp_ipa = 'https://www.beeradvocate.com/lists/style/140/'

# html = urlopen(url_amer_ipa)

response1 = requests.get(url_amer_ipa)
response2 = requests.get(url_ne_ipa)
response3 = requests.get(url_imp_ipa)

In [3]:
# PRE-PROCESS
page1 = response1.text
soup1 = BeautifulSoup(page1, "lxml")
table1 = soup1.findAll("table")

page2 = response2.text
soup2 = BeautifulSoup(page2, "lxml")
table2 = soup2.findAll("table")

page3 = response3.text
soup3 = BeautifulSoup(page3, "lxml")
table3 = soup3.findAll("table")

In [4]:
data_rows1 = table1[0].findAll('tr')[2:]
data_rows2 = table2[0].findAll('tr')[2:]
data_rows3 = table3[0].findAll('tr')[2:]
# data_rows[2] to data_rows[101] contains all of the beers info

In [5]:
# Retrive segment of url link for each specific beer in the top 100 list
# if I were to create a function for this, I would instatiate a dictionary\
# with each key being name of the list, and the value being the list 

amer_ipa_list=[]
for i in range(len(data_rows1)):
    link = data_rows1[i].find('a')['href']
    amer_ipa_list.append(link)

ne_ipa_list=[]
for i in range(len(data_rows2)):
    link = data_rows2[i].find('a')['href']
    ne_ipa_list.append(link)
    
imp_ipa_list=[]
for i in range(len(data_rows3)):
    link = data_rows3[i].find('a')['href']
    imp_ipa_list.append(link)


In [6]:
# check to ensure there are no duplicate beers among the 3 diff. top 100 lists
l1 = set(amer_ipa_list+ne_ipa_list+imp_ipa_list)
len(l1)

300

In [7]:
# concatenate first part and second part of url to get to reviews page of each beer
amer_ipa_fulllist = ['https://www.beeradvocate.com' + amer_ipa_list[i] +'?view=beer&sort=&start=' for i in range(len(amer_ipa_list))]
ne_ipa_fulllist = ['https://www.beeradvocate.com' + ne_ipa_list[i] +'?view=beer&sort=&start=' for i in range(len(ne_ipa_list))]
imp_ipa_fulllist = ['https://www.beeradvocate.com' + imp_ipa_list[i] +'?view=beer&sort=&start=' for i in range(len(imp_ipa_list))]

In [8]:
#Test
amer_ipa_fulllist[0]

'https://www.beeradvocate.com/beer/profile/13014/138735/?view=beer&sort=&start='

### Scraping Part 2
Get reviews for each beer in the Top 100 beer list per category

In [9]:
combined_list = amer_ipa_fulllist+ne_ipa_fulllist+imp_ipa_fulllist

In [30]:
import pickle

with open('beer_list.pkl', 'wb') as f:
    pickle.dump(combined_list, f)
    f.close()

In [32]:
with open('beer_list.pkl', 'rb') as f:
    f_list = pickle.load(f)
    f.close()

In [33]:
# check
print(len(f_list))
f_list[0:5]

300


['https://www.beeradvocate.com/beer/profile/13014/138735/?view=beer&sort=&start=',
 'https://www.beeradvocate.com/beer/profile/22511/160794/?view=beer&sort=&start=',
 'https://www.beeradvocate.com/beer/profile/44221/247726/?view=beer&sort=&start=',
 'https://www.beeradvocate.com/beer/profile/3120/32286/?view=beer&sort=&start=',
 'https://www.beeradvocate.com/beer/profile/30654/262996/?view=beer&sort=&start=']

In [48]:
# check
combined_list[0:5]
len(combined_list)

300

In [593]:
# list_of_reviews =[]
# list_of_reviews2 =[]
# list_of_reviews3 =[]
list_of_reviews =[]


#loop through each link 
# for i in range(len(combined_list)):  # change back to this to scrap all 300 beers
# for i in range(100,175):  # Second batch, list_of_reviews2
# for i in range(175,200):  # Second batch, list_of_reviews3
for i in range(200,300):  # Second batch, list_of_reviews4
    letters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    start = 0
    page_end = 2 #integer representing the last page, 2 is placeholder  
    count = 0
    
    while count <= page_end:
        if count ==0:
            # first access the the inital webpage (start=0)
            new_url = combined_list[i]+'0'
            new_response = requests.get(new_url)
            new_page = new_response.text
            new_soup = BeautifulSoup(new_page, "lxml")
            
            # find number of ratings to determine number of iterations
            rating = new_soup.findAll(class_='ba-ratings')
            rc = rating[0].text
            rating_count = int(rc.replace(',',''))
            # divide rating by 25 and round down to get number of iterations
            ct = rating_count//25 
            # return last page (intger)
            page_end = ct*25
            
            # retrieve reviews on page 1
            review = new_soup.findAll(id='rating_fullview_content_2')
            
            #Add check for # of reviews on page
            for reviewer in range(len(review)):
                rev_comment = list(review[reviewer])
                # check to see if there actually is a review
                if len(rev_comment)>7:
                    list_of_reviews.append(rev_comment)
            count +=25
            
        else:
            # retrieve reviews for rest of the pages
            new_url = combined_list[i]+str(count)
            # add delay to prevent time out
            if count % 500 == 0:
                time.sleep(5)
            if count % 1000 == 0:
                time.sleep(2.5)
            if count % 100 == 0:
                time.sleep(2)

            else:
                try:
                    new_response = requests.get(new_url)
                except:
                    time.sleep(60)
                    try:
                        new_response = requests.get(new_url)
                    except:
                        time.sleep(120)
                        pass #(continue or pass?)
                new_page = new_response.text
                new_soup = BeautifulSoup(new_page, "lxml")
                review = new_soup.findAll(id='rating_fullview_content_2')

                #Add check for # of reviews on page
                for reviewer in range(len(review)):
                    rev_comment = list(review[reviewer])
                    # check to see if there actually is a review
                    if len(rev_comment)>7:
                        # copy from below paste here
                        list_of_reviews.append(rev_comment)
            count +=25    

In [ ]:
# Find index where connection timed out, 99th link
amer_ipa_fulllist.index('https://www.beeradvocate.com/beer/profile/287/1093/?view=beer&sort=&start=')

In [562]:
# Link 0 to 99 
list1 = list_of_reviews
len(list1)

61065

In [564]:
# Link 100 to 175 
list2 = list_of_reviews2
len(list2)

64818

In [582]:
# Link 175 to 200 
list3 = list_of_reviews3
len(list3)

5396

In [594]:
# Link 200 to 300 
list4 = list_of_reviews4
len(list4)

57026

In [595]:
combined_rev_list = list1 + list2 + list3 +list4

In [596]:
len(combined_rev_list)

188305

### Scraping Part 3 
Extract actual reviews from list of html elements

In [597]:
# Extract actual reviews out of list_of_reviews
letters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
str_list_reviews = []

for list_ in combined_rev_list:
    for i in range(len(list_)):
        if str(list_[i])[0] in letters:
            str_list_reviews.append(list_[i])
            

# Implement this into the code above within the loop. 
# Write to csv as I go so I don't lose any data.

In [598]:
#check
print(len(str_list_reviews))
str_list_reviews[47000:47010]

58448


['12 ounce bottle poured into a snifter.',
 'on tap at sams in Durham into a bells pint glass.',
 "Bell's ",
 'Served on tap into tulip glass.',
 'Bottled on 1/23/13 very fresh',
 "Appearance - Beautiful crystal clear copper. I poured it pretty conservatively so there wasn't a great deal of head. Nice lacing.",
 'Big hoppy and malty. Heavy beer. Not very hop forward in the aroma. Sweet. Enjoyable still but not even close to the drinking enjoyment level of hundreds of IPAs being made these days.',
 '12oz. bottle poured into a room-temp pint glass. The bottling date is 1/23/13.',
 'Had never heard of this beer until now. As I understand it is a limited brew, I will continue to order until it runs out. I have to be careful as the 10% ABV knocks me in the dirt pretty quick. With the aroma and taste, it goes down smooth with a pleasant linger on the pallate, which makes it very easy to have too many too quick. There is a nice hint of citrus. Just right.',
 'Hopslam FINALLY became available 

In [599]:
sys.getsizeof(str_list_reviews)

514568

In [ ]:
# import pickle
# import sys
# sys.setrecursionlimit(60000)

# with open('beer_rev_list.pkl', 'wb') as f:
#     pickle.dump(str_list_reviews, f)
#     file.close()

In [ ]:
sys.getsizeof(str_list_reviews)

In [559]:
# testlist=[]
# # retrieve reviews on page 1
# new_url = combined_list[0]+'100'
# new_response = requests.get(new_url)
# new_page = new_response.text
# new_soup = BeautifulSoup(new_page, "lxml")

# review = new_soup.findAll(id='rating_fullview_content_2')

# for reviewer in range(1):
#     rev_comment = list(review[reviewer])
#     # check to see if there actually is a review
#     if len(rev_comment)>=9 and str(rev_comment[8])[0] in letters:
#             testlist.append(rev_comment[8])
        
# len(review)

In [ ]:
# # this returns the reviewer's rating for each category: look,smell,taste, feel, overall
# # however, this is returning different beers
# rating_table = new_soup.findAll(class_="muted")

### Next Steps

1. Tokenization with nltk.tokenize
    * Sentence tokenization
    * word tokenization
    * pre-tokenization:
        * lowercase?
        * stop words?  
        
2. Parts of Speech Tagging
    * from nltk.tag import pos_tag  
    
3. Chunking (extracting phrases) ?
    *Install these?
        * movie_reviews: Imdb reviews characterized as pos & neg
        * treebank: tagged and parsed Wall Street Journal text
        * brown: tagged & categorized English text (news, fiction, etc)
        * from nltk.corpus import treebank_chunk  
        
4. TextBlob Sentiment Analysis
    * gatsby
    * Stemming (optional, depends)
    * Lemmintization (optional, depends)
    * from textblob import TextBlob
    * from nltk.corpus import movie_reviews    
    
5. Bigrams, Trigrams, N-grams (optional?)
    * from nltk.util import ngrams
        from collections import Counter
        from operator import itemgetter
        from nltk.corpus import stopwords    
        
6. CountVectorizer: Convert a collection of text documents to a matrix of token counts This implementation produces a sparse representation. (This step comes after you completed all preprocessing)
    * from sklearn.feature_extraction.text import CountVectorizer
    * from sklearn.feature_extraction.text import TfidfVectorizer 
    * TF: frequency in this document
    * IDF: inverse frequency in the corpus

## RERUN THIS

In [35]:
with open('beer_list.pkl', 'rb') as f:
    combined_list = pickle.load(f)
    f.close()

In [38]:
with open('beerreviews1.csv', 'w') as csvfile:
    csvfile.write('')
    csvfile.close()

In [39]:
# list_of_reviews=[]

#loop through each link 
for i in range(len(combined_list)):  # change back to this to scrap all 300 beers
# for i in range(1):  
    letters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    start = 0
    page_end = 2 #integer representing the last page, 2 is placeholder  
    count = 0
    
    while count <= page_end:
        if count ==0:
            # first access the the inital webpage (start=0)
            new_url = combined_list[i]+'0'
            new_response = requests.get(new_url)
            new_page = new_response.text
            new_soup = BeautifulSoup(new_page, "lxml")
            
            # find number of ratings to determine number of iterations
            rating = new_soup.findAll(class_='ba-ratings')
            rc = rating[0].text
            rating_count = int(rc.replace(',',''))
            # divide rating by 25 and round down to get number of iterations
            ct = rating_count//25 
            # return last page (intger)
            page_end = ct*25
            
            # retrieve reviews on page 1
            review = new_soup.findAll(id='rating_fullview_content_2')
            
            #Add check for # of reviews on page
            for reviewer in range(len(review)):
                rev_comment = list(review[reviewer])
                # check to see if there actually is a review
                if len(rev_comment)>7:
                    for i in range(len(rev_comment)):
                        if str(rev_comment[i])[0] in letters:
                            with open('beerreviews1.csv', 'a') as csvfile:
                                writer=csv.writer(csvfile)
                                writer.writerow([rev_comment[i]])
            count +=25
            
        else:
            # retrieve reviews for rest of the pages
            new_url = combined_list[i]+str(count)
            # add delay to prevent time out
            if count % 500 == 0:
                time.sleep(2)
            if count % 1000 == 0:
                time.sleep(5)
#             if count % 100 == 0:
#                 time.sleep(1)

            else:
                try:
                    new_response = requests.get(new_url)
                except:
                    time.sleep(60)
                    try:
                        new_response = requests.get(new_url)
                    except:
                        time.sleep(120)
                        pass #(continue or pass?)
                new_page = new_response.text
                new_soup = BeautifulSoup(new_page, "lxml")
                review = new_soup.findAll(id='rating_fullview_content_2')

                #Add check for # of reviews on page
                for reviewer in range(len(review)):
                    rev_comment = list(review[reviewer])
                    # check to see if there actually is a review
                    if len(rev_comment)>7:
                        for i in range(len(rev_comment)):
                            if str(rev_comment[i])[0] in letters:
                                with open('beerreviews1.csv', 'a') as csvfile:
                                    writer=csv.writer(csvfile)
                                    writer.writerow([rev_comment[i]])                                
            count +=25    

In [16]:
new_url = combined_list[i]+'0'
new_url

'https://www.beeradvocate.com/beer/profile/38140/238158/?view=beer&sort=&start=0'

In [40]:
df = pd.read_csv('beerreviews1.csv')


,"Citra forward (pineapple, grapefruit, tangerine etc), mild bitter finish, it is good but doesn’t really stand out from the crowd though that said it is still quietly very good"
0,What?!?! This beer is hop magic!
1,"The stamp is a bit smudged, but it looks like ..."
2,Wow isn't that the most meanest & toughest bee...
3,"Aggressive but good. Citrus aroma, nice color ..."
4,There is a reason Beer Connoisseur rated this ...


In [41]:
df.shape

(9679, 1)

Modify & Rerun orginal Scraper

In [ ]:
with open('beerreviews2.1.csv', 'w') as csvfile:
    csvfile.write('')
    csvfile.close()

In [43]:
# list_of_reviews=[]

#loop through each link 
for i in range(len(combined_list[0:100])):  # change back to this to scrap all 300 beers
# for i in range(1):  
    letters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    start = 0
    page_end = 2 #integer representing the last page, 2 is placeholder  
    count = 0
    
    while count <= page_end:
        if count ==0:
            # first access the the inital webpage (start=0)
            new_url = combined_list[i]+'0'
            new_response = requests.get(new_url)
            new_page = new_response.text
            new_soup = BeautifulSoup(new_page, "lxml")
            
            # find number of ratings to determine number of iterations
            rating = new_soup.findAll(class_='ba-ratings')
            rc = rating[0].text
            rating_count = int(rc.replace(',',''))
            # divide rating by 25 and round down to get number of iterations
            ct = rating_count//25 
            # return last page (intger)
            page_end = ct*25
            
            # retrieve reviews on page 1
            review = new_soup.findAll(id='rating_fullview_content_2')
            
            #Add check for # of reviews on page
            for reviewer in range(len(review)):
                rev_comment = list(review[reviewer])
                # check to see if there actually is a review
                if len(rev_comment)>7:
                    for i in range(len(rev_comment)):
                        if str(rev_comment[i])[0] in letters:
                            with open('beerreviews2.1.csv', 'a') as csvfile:
                                writer=csv.writer(csvfile)
                                writer.writerow([rev_comment[i]])
            count +=25
            
        else:
            # retrieve reviews for rest of the pages
            new_url = combined_list[i]+str(count)
            # add delay to prevent time out
            if count % 500 == 0:
                time.sleep(2)
            if count % 1000 == 0:
                time.sleep(3)
#             if count % 100 == 0:
#                 time.sleep(1)

            else:
                try:
                    new_response = requests.get(new_url)
                except:
                    time.sleep(30)
                    try:
                        new_response = requests.get(new_url)
                    except:
                        time.sleep(60)
                        pass 
                new_page = new_response.text
                new_soup = BeautifulSoup(new_page, "lxml")
                review = new_soup.findAll(id='rating_fullview_content_2')

                #Add check for # of reviews on page
                for reviewer in range(len(review)):
                    rev_comment = list(review[reviewer])
                    # check to see if there actually is a review
                    if len(rev_comment)>7:
                        for i in range(len(rev_comment)):
                            if str(rev_comment[i])[0] in letters:
                                with open('beerreviews2.1.csv', 'a') as csvfile:
                                    writer=csv.writer(csvfile)
                                    writer.writerow([rev_comment[i]])                                
            count +=25    

In [44]:
df2 = pd.read_csv('beerreviews2.1.csv')
len(df2)

3124

In [45]:
with open('beerreviews3.1.csv', 'w') as csvfile:
    csvfile.write('')
    csvfile.close()

In [46]:
# list_of_reviews =[]
# list_of_reviews2 =[]
# list_of_reviews3 =[]
# list_of_reviews =[]


#loop through each link 
for i in range(0,100):  # Second batch, list_of_reviews4
    letters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    start = 0
    page_end = 2 #integer representing the last page, 2 is placeholder  
    count = 0
    
    while count <= page_end:
        if count ==0:
            # first access the the inital webpage (start=0)
            new_url = combined_list[i]+'0'
            new_response = requests.get(new_url)
            new_page = new_response.text
            new_soup = BeautifulSoup(new_page, "lxml")
            
            # find number of ratings to determine number of iterations
            rating = new_soup.findAll(class_='ba-ratings')
            rc = rating[0].text
            rating_count = int(rc.replace(',',''))
            # divide rating by 25 and round down to get number of iterations
            ct = rating_count//25 
            # return last page (intger)
            page_end = ct*25
            
            # retrieve reviews on page 1
            review = new_soup.findAll(id='rating_fullview_content_2')
            
            #Add check for # of reviews on page
            for reviewer in range(len(review)):
                rev_comment = list(review[reviewer])
                # check to see if there actually is a review
                if len(rev_comment)>7:
                    for i in range(len(rev_comment)):
                        if str(rev_comment[i])[0] in letters:
                            with open('beerreviews3.1.csv', 'a') as csvfile:
                                writer=csv.writer(csvfile)
                                writer.writerow([rev_comment[i]])
            count +=25
            
        else:
            # retrieve reviews for rest of the pages
            new_url = combined_list[i]+str(count)
            # add delay to prevent time out
            if count % 500 == 0:
                time.sleep(1)
            if count % 2000 == 0:
                time.sleep(2)
#             if count % 100 == 0:
#                 time.sleep(2)

#             else:
#                 try:
                new_response = requests.get(new_url)
#                 except:
#                     time.sleep(60)
#                     try:
#                         new_response = requests.get(new_url)
#                     except:
#                         time.sleep(120)
#                         pass #(continue or pass?)
                new_page = new_response.text
                new_soup = BeautifulSoup(new_page, "lxml")
                review = new_soup.findAll(id='rating_fullview_content_2')

                #Add check for # of reviews on page
                for reviewer in range(len(review)):
                    rev_comment = list(review[reviewer])
                    # check to see if there actually is a review
                    if len(rev_comment)>7:
                        for i in range(len(rev_comment)):
                            if str(rev_comment[i])[0] in letters:
                                with open('beerreviews3.1.csv', 'a') as csvfile:
                                    writer=csv.writer(csvfile)
                                    writer.writerow([rev_comment[i]]) 
            count +=25    

In [47]:
df3 = pd.read_csv('beerreviews3.1.csv')
len(df3)

579

In [66]:
df4 = pd.read_csv('beerreviewsv2.csv',header=None)
len(df4)

3129

In [67]:
df4.drop_duplicates(inplace=True)

In [68]:
df4.head()
len(df4)

687

In [81]:
df4.loc[2592][0]

'On Tap at Mike Taproom Largo,FL Pours a golden color with a fresh small white head. Smell was floral,citrus and dank hop. Taste followed nose was crisp with hops and citrus forward . Mouth feel crisp with some pine bitterness in finish but floral/tropical hops are in charge here. A delightful IPA.'

In [89]:
df5 = pd.read_csv('beerreviewsv2.1.csv',header=None)
len(df5)

15840

In [94]:
df5.loc[10][0]

'It’s spiritual!!! Listen to your favorite music and prepare to enjoy!!!!!'